In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv("../input/train.tsv", sep="\t")
df_test = pd.read_csv("../input/test.tsv", sep="\t")

def transform(df, train=True):
    NUM_FOLDS = 5
    df["phrase_count"] = df.groupby("SentenceId")["Phrase"].transform("count")
    df["word_count"] = df["Phrase"].apply(lambda x: len(x.split()))
    df["has_upper"] = df["Phrase"].apply(lambda x: x.lower() != x)
    df["sentence_end"] = df["Phrase"].apply(lambda x: x.endswith("."))
    df["after_comma"] = df["Phrase"].apply(lambda x: x.startswith(","))
    df["sentence_start"] = df["Phrase"].apply(lambda x: "A" <= x[0] <= "Z")
    if train:
        df["fold_id"] = df["SentenceId"].apply(lambda x: x%NUM_FOLDS)
    return df

df = transform(df)
df_test = transform(df_test, train=False)

In [ ]:
from sklearn.utils import shuffle
from keras.utils import to_categorical

df = shuffle(df, random_state=42)
#X = df['Phrase']
#y = df['Sentiment']

#y = to_categorical(y)

In [ ]:
from keras.preprocessing.text import Tokenizer

max_features=15000
maxlen = 57

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(df['Phrase']))

In [ ]:
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from keras.models import Model
from keras.layers import Input, Dense, Embedding, MaxPooling1D, Conv1D, SpatialDropout1D
from keras.layers import add, Dropout, PReLU, BatchNormalization, GlobalMaxPooling1D, concatenate
from keras.preprocessing import text, sequence
from keras.callbacks import Callback, EarlyStopping
from keras import optimizers
from keras import initializers, regularizers, constraints, callbacks
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
max_features=15000
maxlen = 57

seq = tokenizer.texts_to_sequences(df['Phrase'])
seq = pad_sequences(seq, maxlen=maxlen)
seq_test = tokenizer.texts_to_sequences(df_test['Phrase'])
seq_test = pad_sequences(seq_test, maxlen=maxlen)

In [ ]:
def build_model():
    filter_nr = 64
    filter_size = 3
    max_pool_size = 3
    max_pool_strides = 2
    dense_nr = 256
    spatial_dropout = 0.2
    dense_dropout = 0.5
    train_embed = False
    conv_kern_reg = regularizers.l2(0.00001)
    conv_bias_reg = regularizers.l2(0.00001)

    inp1 = Input(shape=(6, ))
    dense_vector = BatchNormalization()(inp1)

    comment = Input(shape=(maxlen,))
    emb_comment = Embedding(max_features, 64, input_length=maxlen)(comment)
    emb_comment = SpatialDropout1D(spatial_dropout)(emb_comment)

    block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(emb_comment)
    block1 = BatchNormalization()(block1)
    block1 = PReLU()(block1)
    block1 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block1)
    block1 = BatchNormalization()(block1)
    block1 = PReLU()(block1)

    block1_output = add([block1, emb_comment])
    block1_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block1_output)

    block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block1_output)
    block2 = BatchNormalization()(block2)
    block2 = PReLU()(block2)
    block2 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block2)
    block2 = BatchNormalization()(block2)
    block2 = PReLU()(block2)

    block2_output = add([block2, block1_output])
    block2_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block2_output)

    block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block2_output)
    block3 = BatchNormalization()(block3)
    block3 = PReLU()(block3)
    block3 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block3)
    block3 = BatchNormalization()(block3)
    block3 = PReLU()(block3)

    block3_output = add([block3, block2_output])
    block3_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block3_output)

    block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block3_output)
    block4 = BatchNormalization()(block4)
    block4 = PReLU()(block4)
    block4 = Conv1D(filter_nr, kernel_size=filter_size, padding='same', activation='linear', 
                kernel_regularizer=conv_kern_reg, bias_regularizer=conv_bias_reg)(block4)
    block4 = BatchNormalization()(block4)
    block4 = PReLU()(block4)

    block4_output = add([block4, block3_output])
    block4_output = MaxPooling1D(pool_size=max_pool_size, strides=max_pool_strides)(block4_output)

    output = GlobalMaxPooling1D()(block4_output)
    output = concatenate([dense_vector, output])
    output = Dense(50, activation="relu")(output)
    output = Dense(20, activation="relu")(output)
    output = Dense(5, activation='softmax')(output)

    model = Model([inp1, comment], output)
    return model

In [ ]:
from keras.callbacks import ModelCheckpoint
dense_features = ["phrase_count", "word_count", "has_upper", "after_comma", "sentence_start", "sentence_end"]
epochs = 15
batch_size = 1024
test_preds = np.zeros((df_test.shape[0], 5))

for i in range(5):
    print("FOLD"+str(i+1))
    train_seq, val_seq = seq[df["fold_id"] != i], seq[df["fold_id"] == i]
    train_dense, val_dense = df[df["fold_id"] != i][dense_features], df[df["fold_id"] == i][dense_features]
    early_stopping = EarlyStopping(monitor="val_acc", patience=3, verbose=1)
    y_train = to_categorical(df[df["fold_id"] != i]["Sentiment"])
    y_val = to_categorical(df[df["fold_id"] == i]["Sentiment"])
    early_stopping = EarlyStopping(monitor="val_acc", patience=2, verbose=1)
    model = build_model()
    model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(), metrics=['accuracy'])
    model.fit([train_dense, train_seq],y_train, validation_data=([val_dense, val_seq], y_val), epochs=epochs, batch_size=batch_size, 
              shuffle=True,callbacks=[early_stopping], verbose=1)
    test_preds += model.predict([df_test[dense_features], seq_test], batch_size=1024, verbose=1)
test_preds /= 5

In [ ]:
df_test['Sentiment'] = np.argmax(test_preds, axis=1)
df_test[["PhraseId", "Sentiment"]].to_csv("submittion.csv", index=False)

In [ ]:
df_test
